<a href="https://colab.research.google.com/github/Isac999/Auto-Machine-Learining-Project/blob/main/autokeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://www.dropbox.com/s/xzyqgmb60lzsf88/USD_BRL2010_2021.csv'

--2022-02-25 17:56:47--  https://www.dropbox.com/s/xzyqgmb60lzsf88/USD_BRL2010_2021.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xzyqgmb60lzsf88/USD_BRL2010_2021.csv [following]
--2022-02-25 17:56:47--  https://www.dropbox.com/s/raw/xzyqgmb60lzsf88/USD_BRL2010_2021.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6941808eb150609524595d1378.dl.dropboxusercontent.com/cd/0/inline/BgaCNyuhTMpS1hvMD1COb6vR1djvSz5p4I8ci5ElnWrqHpj-x29-I3EiVgxXpK4cGRskz3kygtes_cgFsDfUyKb9ByT-VZt25IJsnJpD1Pk_LizPpxaazw-JV8pWIXZk-CVFXm2bmZGdoVJy9nfDq9AS/file# [following]
--2022-02-25 17:56:47--  https://uc6941808eb150609524595d1378.dl.dropboxusercontent.com/cd/0/inline/BgaCNyuhTMpS1hvMD1COb6vR1djvSz5p4I8ci5ElnWrqHpj-x29-I3EiVgxXpK4cGR

In [ ]:
!pip install pandas
!pip install autokeras

     |████████████████████████████████| 160 kB 26.6 MB/s 
     |████████████████████████████████| 98 kB 6.4 MB/s 
     |████████████████████████████████| 462 kB 47.5 MB/s 


In [ ]:
import pandas as pd
import autokeras as ak

In [ ]:
import datetime

In [ ]:
df = pd.read_csv('/content/USD_BRL2010_2021.csv')

In [ ]:
df.shape

(3118, 6)

In [ ]:
df['Var%'] = df['Var%'].apply(lambda x : float(x.replace('%', '').replace(',','.')))
df['Último'] = df['Último'].apply(lambda x : float(x.replace(',', '.')))
df['Abertura'] = df['Abertura'].apply(lambda x : float(x.replace(',', '.')))
df['Máxima'] = df['Máxima'].apply(lambda x : float(x.replace(',', '.')))
df['Mínima'] = df['Mínima'].apply(lambda x : float(x.replace(',', '.')))

In [ ]:
df['Ano'] = df['Data'].apply(lambda x: int(x.split('.')[-1]))
df['Mes'] = df['Data'].apply(lambda x: x.split('.')[1])
df['Semana'] = df['Data'].apply(lambda x: int(
                                datetime.date(
                                  int(x.split('.')[-1]),
                                  int(x.split('.')[1]),
                                  int(x.split('.')[0])
                                  ).strftime("%V")
))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      3118 non-null   object 
 1   Último    3118 non-null   float64
 2   Abertura  3118 non-null   float64
 3   Máxima    3118 non-null   float64
 4   Mínima    3118 non-null   float64
 5   Var%      3118 non-null   float64
 6   Ano       3118 non-null   int64  
 7   Mes       3118 non-null   object 
 8   Semana    3118 non-null   int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 219.4+ KB


In [ ]:
target = []
for i in df.index:
  if i > 1:
    target.append(df['Último'][i-1])
  else:
    target.append(df['Último'][0])
df['Target'] = target

In [ ]:
df = df.sort_index(ascending=False)
df = df.reset_index()
df.drop(['index'], axis=1, inplace=True)
df.head()

,Data,Último,Abertura,Máxima,Mínima,Var%,Ano,Mes,Semana,Target
0,01.01.2010,1.7430,1.7430,1.7430,1.7430,0.00,2010,01,53,1.7212
1,04.01.2010,1.7212,1.7288,1.7338,1.7161,-1.25,2010,01,1,1.7310
2,05.01.2010,1.7310,1.7193,1.7398,1.7145,0.57,2010,01,1,1.7325
3,06.01.2010,1.7325,1.7296,1.7393,1.7241,0.09,2010,01,1,1.7465
4,07.01.2010,1.7465,1.7310,1.7518,1.7310,0.81,2010,01,1,1.7262


In [ ]:
df.head()

,Data,Último,Abertura,Máxima,Mínima,Var%,Ano,Mes,Semana,Target
0,21.12.2021,5.7456,5.7417,5.7567,5.7119,0.05,2021,12,51,5.7456
1,20.12.2021,5.7425,5.7141,5.7457,5.6899,0.81,2021,12,51,5.7456
2,17.12.2021,5.6961,5.6887,5.7167,5.6610,0.14,2021,12,50,5.7425
3,16.12.2021,5.6884,5.6693,5.7256,5.6589,0.14,2021,12,50,5.6961
4,15.12.2021,5.6806,5.6873,5.7364,5.6574,-0.02,2021,12,50,5.6884


In [ ]:
x_train = df.loc[df['Ano'] < 2021].drop(['Data', 'Target'], axis=1)
y_train = df.loc[df['Ano'] < 2021]['Target']

x_test = df.loc[df['Ano'] >= 2021].drop(['Data', 'Target'], axis=1)
y_test = df.loc[df['Ano'] >= 2021]['Target']

In [ ]:
model = ak.StructuredDataRegressor(overwrite=True, max_trials=10)
model.fit(x_train, y_train, epochs=10)

Trial 10 Complete [00h 00m 06s]
val_loss: 0.6990402936935425

Best val_loss So Far: 0.3079054057598114
Total elapsed time: 00h 01m 02s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
90/90 [==============================] - 1s 2ms/step - loss: 3.3841 - mean_squared_error: 3.3841
Epoch 2/10
90/90 [==============================] - 0s 2ms/step - loss: 1.4760 - mean_squared_error: 1.4760
Epoch 3/10
90/90 [==============================] - 0s 3ms/step - loss: 0.7397 - mean_squared_error: 0.7397
Epoch 4/10
90/90 [==============================] - 0s 2ms/step - loss: 0.5721 - mean_squared_error: 0.5721
Epoch 5/10
90/90 [==============================] - 0s 3ms/step - loss: 0.4322 - mean_squared_error: 0.4322
Epoch 6/10
90/90 [==============================] - 0s 3ms/step - loss: 0.3971 - mean_squared_error: 0.3971
Epoch 7/10
90/90 [==============================] - 0s 3ms/step - loss: 0.3528 - mean_squared_error: 0.3528
Epoch 8/10
90/90 [==============================] - 0s 2ms/step - loss:

In [ ]:
loss, val_loss = model.evaluate(x=x_test, y=y_test)
print('val_loss: %.3f' % val_loss, loss)

8/8 [==============================] - 0s 3ms/step - loss: 0.0291 - mean_squared_error: 0.0291
val_loss: 0.029 0.029072513803839684


In [ ]:
predicted_y = model.predict(x_test)
x_test['predict'] = predicted_y
x_test['Target'] = y_test

8/8 [==============================] - 0s 3ms/step


In [ ]:
x_test

,Último,Abertura,Máxima,Mínima,Var%,Ano,Mes,Semana,predict,Target
2867,5.1935,5.1934,5.1935,5.1934,0.00,2021,01,53,5.334915,5.2968
2868,5.2968,5.1351,5.2972,5.1197,1.99,2021,01,1,5.451208,5.2854
2869,5.2854,5.2955,5.3547,5.2515,-0.22,2021,01,1,5.312984,5.3139
2870,5.3139,5.2863,5.3597,5.2318,0.54,2021,01,1,5.383051,5.4101
2871,5.4101,5.3169,5.4187,5.2989,1.81,2021,01,1,5.566402,5.4178
...,...,...,...,...,...,...,...,...,...,...
3113,5.6806,5.6873,5.7364,5.6574,-0.02,2021,12,50,5.645349,5.6884
3114,5.6884,5.6693,5.7256,5.6589,0.14,2021,12,50,5.648309,5.6961
3115,5.6961,5.6887,5.7167,5.6610,0.14,2021,12,50,5.661537,5.7425
3116,5.7425,5.7141,5.7457,5.6899,0.81,2021,12,51,5.802853,5.7456
